### Import Libraries

In [1]:
# Data-Interchange Format
import json
# Binary Protocol
import pickle
# Zip Files
import zipfile

# TensorFlow Installation
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Data Manipulation
import pandas as pd

## Descompressing Original Database

In [4]:
# Database Path Address
local_zip = "/content/dataset_original.zip"
# Decrompress & Reading Process
zip_ref = zipfile.ZipFile(local_zip, "r")
# Extracting & Storaging Process
zip_ref.extractall("dataset_original")
# Closing
zip_ref.close()

# JSON to CSV
##  Defining JSON FILE Route 

Process applied on Training & Test 


In [7]:
# Training Type
type_file = "test"
# File path
path = "/content/test.json"
# Opeing Process
data_file = open(path)
# Retrieving from
data = json.load(data_file)

#### Let's difine the source JSON structure and the relevant variables that we must save within the CSV

In [8]:
csv_list = []
'''
Let's Perambulate on JSON file structure, extracting relevant variable 
1. Class
2. Bounding box (x initial, y initial, x final, y final)
3. Image Width & Height.
4. File Name.
'''
# Perambulate procedure
for classification in data:
  # Width & Height Classification
  width, height = classification['width'], classification['height']
  # Image Address Classification 
  image = classification['image']
  # Iteration on
  for item in classification['tags']:
    name = item['name']
    xmin = item['pos']['x']
    ymin = item['pos']['y']
    xmax = item['pos']['x'] + item['pos']['w']
    ymax = item['pos']['y'] + item['pos']['h']

    # Data Allocation
    value = (image, width, height, name, xmin, ymin, xmax, ymax)
    # Adding & storage 
    csv_list.append(value)

# At the end, Let's store it within a CSV file named on the following columns 
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
# CSV DataFrame
csv_df = pd.DataFrame(csv_list, columns = column_name)

# Convert the Dataframe a CSV
csv_df.to_csv("/content/{}_labels.csv".format(type_file))

# From CSV to TFRecord
### Installing TensorFlow Object Detection library 

In [9]:
!pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.1 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844510 sha256=0a0c0789daf13778785ebcad87ac5954e28d3097b10b28e91fdf1ddbda165719
  Stored in directory: /root/.cache/pip/wheels/38/6f/73/3c0c70f9a8219c31c96c495b686af1416154b60707fd96a4cd
Successfully built tensorflow-object-detection-api


In [10]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Note: switching to '58d19c67e1d30d905dd5c6e5092348658fed80af'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 58d19c67e Internal change
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0

### Let's take the main base script & modify it to this project purpose

In [14]:
# '''
# Retrieved from Main website: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/tensorflow-1.14/training.html

# '''
# from __future__ import division
# from __future__ import print_function
# from __future__ import absolute_import

# import os
# import io
# import pandas as pd
# import tensorflow as tf
# import sys
# sys.path.append("../../models/research")

# from PIL import Image
# from object_detection.utils import dataset_util
# from collections import namedtuple, OrderedDict


# # Replacecing the model fixed flags(class) with the project class 
# # New names were set followwing TensorFlow Object Detection Convention.
# def class_text_to_int(row_label):
#     if row_label == "Car": 
#       return 1
#     elif row_label == "Motorcycle":
#       return 2
#     else:
#         None


# def split(df, group):
#     data = namedtuple('data', ['filename', 'object'])
#     gb = df.groupby(group)
#     return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


# def create_tf_example(group, path):
#   with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
#     encoded_jpg = fid.read()
#     encoded_jpg_io = io.BytesIO(encoded_jpg)
#     image = Image.open(encoded_jpg_io)
#     width, height = image.size

#     filename = group.filename.encode('utf8')
#     image_format = b'jpg'
#     # check if the image format is matching with your images.
#     xmins = []
#     xmaxs = []
#     ymins = []
#     ymaxs = []
#     classes_text = []
#     classes = []

#     for index, row in group.object.iterrows():
#       xmins.append(row['xmin'] / width)
#       xmaxs.append(row['xmax'] / width)
#       ymins.append(row['ymin'] / height)
#       ymaxs.append(row['ymax'] / height)
#       classes_text.append(row['class'].encode('utf8'))
#       classes.append(class_text_to_int(row['class']))

#     tf_example = tf.train.Example(features=tf.train.Features(feature={
#         'image/height': dataset_util.int64_feature(height),
#         'image/width': dataset_util.int64_feature(width),
#         'image/filename': dataset_util.bytes_feature(filename),
#         'image/source_id': dataset_util.bytes_feature(filename),
#         'image/encoded': dataset_util.bytes_feature(encoded_jpg),
#         'image/format': dataset_util.bytes_feature(image_format),
#         'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
#         'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
#         'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
#         'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
#         'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
#         'image/object/class/label': dataset_util.int64_list_feature(classes),
#     }))
#     return tf_example

# # Since this model were perfored on Google Colab
# # I Modefied the "Main Entry Point " file location 
# # Similar Google Colab URL where the folder is coming from.
# # On Read.me & requirements I've left all the files neccesary to perform this project
# output_path = "train.record"
# # Image Address
# image_dir = "/content/dataset_original/dataset_final"
# # CSV Folder
# csv_input = "/content/train_labels.csv"

# writer = tf.io.TFRecordWriter(output_path)
# path = os.path.join(image_dir)
# examples = pd.read_csv(csv_input)
# grouped = split(examples, 'filename')
# for group in grouped:
#     tf_example = create_tf_example(group, path)
#     writer.write(tf_example.SerializeToString())

# writer.close()
# output_path = os.path.join(os.getcwd(), output_path)
# print('Successfully created the TFRecords: {}'.format(output_path))